<a href="https://colab.research.google.com/github/AhmadElsayed-ML/RNN-Play-Generator/blob/main/RNN_Play_Genarator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
#Read then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
#length of text is the number of charecters in it
print('lenght of text:{} characters'.format(len(text)))

lenght of text:1115394 characters


In [ ]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
vocab = sorted(set(text))
#creating a mapping from unique characters to indices
char2idx= {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
#lets look at how part of our text is encoded
print('text:',text[:13])
print('Encoded:', text_to_int(text[:13]))

text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])
print(int_to_text(text_as_int[:13]))

First Citizen


In [ ]:
seq_lenght = 100 #lenght of sequence for a training example
examples_per_epoch = len(text)//(seq_lenght+1)

#Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_lenght+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk): #for the example : hello
  input_text = chunk[:-1] #hell
  target_text = chunk[1:] #ello
  return input_text , target_text #hell , ello
dataset = sequences.map(split_input_target) #we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("input")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))




EXAMPLE

input
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

input
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) #vocba is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

#Buffer size to shuffle the dataset
#(TF data is designed to work with possibly infinite sequences)
#so it dosen't attempt to shuffle the entire sequence in memory .
#Insted it maintains a buffer in wich it shuffles elements)
BUFFER_SIZE = 10000
data= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

